In [0]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg as LA
import os
from copy import deepcopy

For calculating $F$, for a given $A$, we use 

$Af=0$

where,

 $f=[f_{11}, f_{12}, f_{13}, f_{21}, f_{22}, f_{23}, f_{31}, f_{32}, f_{33}]^T$

 Then, we do $SVD$ of $A$,
 
 As, $A=UDV^T$

 and $f$ is in null space of $A$, we pick last vector of $V$.

 $f=v_8$

 We enforce rank 2 of F by making last element of $D'$ equal to 0, where,

 $F_{estimated}=U'*D'*V'^{T}$

In [0]:
''' This function estimates F matrix for a given A matrix '''

def estimateF(A):
    u,d,v=LA.svd(A)
    # print(v.shape)
    F=v[8,:].reshape(3,3)
    # print(F.shape)
    u,d,v=LA.svd(F)
    d[-1]=0
    F=u.dot(np.diag(d).dot(v))
    return F

For RANSAC, we use the equation 

$x_{1}^{T}*F*x_2=0$

Or,


$(x_1 \otimes x_2)*f=0$

If, this product is less than a certain threshold, we call it aws an inlier, otherwise it is an outlier.


In [0]:
import random

def ransac(impts1, impts2,N1,N2):
    if len(impts1>=8):
        print(len(impts1))
        thresh=1e-4
        maxinliers=0
        permInliers=[]
        pointsind=[]
        for i in range (300):
            rndind = random.sample(range(len(impts1)),8)
            temp=np.array([]).reshape(0,9)
            for ind in rndind:
                # print(np.kron(impts1[ind],impts2[ind]).shape)
                temp=np.vstack((temp,np.array([impts1[ind,0]*impts2[ind,0],impts1[ind,0]*impts2[ind,1],impts1[ind,0]*impts2[ind,2],
                                               impts1[ind,1]*impts2[ind,0],impts1[ind,1]*impts2[ind,1],impts1[ind,1]*impts2[ind,2],
                                               impts2[ind,0], impts2[ind,1], impts2[ind,2]])))
                # print('temp',temp)
            Ftemp=estimateF(temp) 
            errmat=np.abs(impts1.dot(Ftemp.dot(impts2.T)))
            inliers=errmat.diagonal() <= thresh
            noOfInliers=np.sum(inliers)
            #print('inliers',noOfInliers)
            if noOfInliers > maxinliers:
                F=deepcopy(Ftemp)
                maxinliers=noOfInliers
                permInliers=inliers
                pointsind=rndind
            if maxinliers>=0.6*len(impts1):
                break
        temp=np.array([]).reshape(0,9)
        print('inliers ',maxinliers)
        for i in range(len(permInliers)):
            if permInliers[i]:
                temp=np.vstack((temp,np.array([impts1[i,0]*impts2[i,0],impts1[i,0]*impts2[i,1],impts1[i,0]*impts2[i,2],
                                               impts1[i,1]*impts2[i,0],impts1[i,1]*impts2[i,1],impts1[i,1]*impts2[i,2],
                                               impts2[i,0], impts2[i,1], impts2[i,2]])))
        for ind in pointsind:
            temp=np.vstack((temp,np.array([impts1[ind,0]*impts2[ind,0],impts1[ind,0]*impts2[ind,1],impts1[ind,0]*impts2[ind,2],
                                               impts1[ind,1]*impts2[ind,0],impts1[ind,1]*impts2[ind,1],impts1[ind,1]*impts2[ind,2],
                                               impts2[ind,0], impts2[ind,1], impts2[ind,2]])))
        F=estimateF(temp)
        return (N2.T).dot(F.dot(N1))
        #return F

In [0]:
def read(txtfile):
    line=txtfile.readline()
    # print(line)
    line=line.split()
    groundC=list(map(float,line))
    groundC=np.vstack((np.array(groundC).reshape(3,4),np.array([0,0,0,1])))
    #print(groundC)
    return groundC

In [0]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
# from ransac import *
# import os
file2 = open("your-result.txt","w")
file2.close()
C=np.eye(4)
file1 = open("ground-truth.txt","r") 
file2 = open("your-result.txt","a")
file2.write('1.000000e+00 -1.822835e-10 5.241111e-10 -5.551115e-17 -1.822835e-10 9.999999e-01 -5.072855e-10 -3.330669e-16 5.241111e-10 -5.072855e-10 9.999999e-01 2.220446e-16')
file2.write('\n')
groundC=read(file1)
groundtLast=groundC[0:3,3]
folder='./image_0/'
numberOfimages=1100
for image in range(numberOfimages):
    print (image)
    if image==0:
        img1=cv2.imread(folder+str(image).zfill(6)+'.png')
        img2=cv2.imread(folder+str(image+1).zfill(6)+'.png')
        # plt.imshow(img1)
        # print(img1.shape)
    else:
        img1=deepcopy(img2)
        img2=cv2.imread(folder+str(image+1).zfill(6)+'.png')  
    # detector = cv2.SIFT()
    orb = cv2.ORB_create()
    Keypoints1, Descriptors1 = orb.detectAndCompute(img1, None)
    Keypoints2, Descriptors2 = orb.detectAndCompute(img2, None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(Descriptors1,Descriptors2)
    matches = sorted(matches, key = lambda x:x.distance)
    im1pts=np.array([]).reshape(0,2)
    im2pts=np.array([]).reshape(0,2)
    for m in matches:
        img1Idx = m.queryIdx
        img2Idx = m.trainIdx
        (img1x, img1y) = Keypoints1[img1Idx].pt
        (img2x, img2y) = Keypoints2[img2Idx].pt
        # print(img1x,img1y)
        # print(img2x,img2y)
        im1pts=np.vstack((im1pts,np.array([img1x,img1y])))
        im2pts=np.vstack((im2pts,np.array([img2x,img2y])))

    points1=deepcopy(im1pts)
    points2=deepcopy(im2pts)
    im1pts=np.hstack((im1pts,np.ones((im1pts.shape[0],1))))
    im2pts=np.hstack((im2pts,np.ones((im1pts.shape[0],1))))
    N=np.array([[2/375,0,-1],[0,2/1242,-1],[0,0,1]])
    im1pts=(N.dot(im1pts.T)).T
    im2pts=(N.dot(im2pts.T)).T

    # print(len(im1pts))
    F=ransac(im2pts,im1pts,N,N)
    
#     f = 7.215377000000e+02
#     pp = (6.095593000000e+02,1.728540000000e+02)
#     E = cv2.findEssentialMat(points1, points2,f, pp)
#     K = np.array([[7.215377000000e+02,0.000000000000e+00,6.095593000000e+02],
#               [0.000000000000e+00,7.215377000000e+02,1.728540000000e+02],
#               [0.000000000000e+00,0.000000000000e+00,1.000000000000e+00]])
    K = np.array([[7.070912000000e+02, 0.000000000000e+00, 6.018873000000e+02], [0.000000000000e+00, 7.070912000000e+02, 1.831104000000e+02], [0.000000000000e+00, 0.000000000000e+00, 1.000000000000e+00]])  
    
    
    E=(K.T).dot(F.dot(K))
    u,d,v=LA.svd(E)
    E=u.dot(np.diag([1,1,0]).dot(v))
    # print(E)
    A=cv2.recoverPose(E,points1,points2,K)
    #print('E:')
    #print(E)
#     _,R,t,_, = cv2.recoverPose(E[0],points1,points2,K,100)
    R=A[1]
    t=A[2]
    groundC=read(file1)
    groundt=groundC[0:3,3]
    t=t*LA.norm(groundt-groundtLast)
    groundtLast=deepcopy(groundt)
    T=np.vstack((np.hstack((R,t)),np.array([0,0,0,1])))
    C=C.dot(T)
    #print('C>>> ',C)
    X = C/C[3,3]
    
    # print('#####gt>>',groundt)
#     print(C)
    #print ('R-->',R)
    #print ('t-->',t)
    X = X.flatten()
#     file2 = open("your-result.txt","a")
    for i in range(12):
        if i==11:
            file2.write(str(X.item(i)))
        else:    
            file2.write(str(X.item(i)))
            file2.write(" ")
    

    file2.write('\n')
file2.close()

0
302
inliers  128
1
315
inliers  131
2
320
inliers  142
3
317
inliers  141
4
306
inliers  103
5
308
inliers  97
6
299
inliers  92
7
314
inliers  64
8
299
inliers  61
9
284
inliers  108
10
300
inliers  108
11
309
inliers  114
12
311
inliers  127
13
300
inliers  113
14
293
inliers  105
15
296
inliers  107
16
322
inliers  121
17
290
inliers  76
18
286
inliers  79
19
294
inliers  83
20
299
inliers  96
21
324
inliers  128
22
326
inliers  146
23
344
inliers  187
24
340
inliers  175
25
325
inliers  150
26
332
inliers  143
27
330
inliers  165
28
327
inliers  160
29
332
inliers  200
30
323
inliers  168
31
346
inliers  183
32
347
inliers  173
33
338
inliers  166
34
344
inliers  198
35
338
inliers  186
36
348
inliers  159
37
339
inliers  155
38
326
inliers  136
39
336
inliers  146
40
324
inliers  134
41
341
inliers  133
42
316
inliers  134
43
304
inliers  107
44
312
inliers  119
45
311
inliers  103
46
314
inliers  115
47
309
inliers  117
48
318
inliers  130
49
322
inliers  94
50
310
inliers  124

319
inliers  114
400
356
inliers  127
401
331
inliers  135
402
330
inliers  121
403
321
inliers  124
404
328
inliers  149
405
334
inliers  121
406
318
inliers  117
407
313
inliers  95
408
326
inliers  95
409
325
inliers  86
410
299
inliers  105
411
319
inliers  72
412
311
inliers  121
413
317
inliers  125
414
311
inliers  108
415
318
inliers  123
416
317
inliers  70
417
344
inliers  163
418
361
inliers  151
419
370
inliers  104
420
368
inliers  176
421
351
inliers  125
422
357
inliers  187
423
344
inliers  155
424
331
inliers  117
425
328
inliers  105
426
315
inliers  110
427
312
inliers  105
428
320
inliers  141
429
322
inliers  136
430
342
inliers  133
431
352
inliers  125
432
356
inliers  114
433
368
inliers  127
434
367
inliers  130
435
361
inliers  132
436
339
inliers  183
437
339
inliers  70
438
324
inliers  73
439
337
inliers  160
440
363
inliers  130
441
359
inliers  196
442
373
inliers  86
443
380
inliers  211
444
374
inliers  143
445
374
inliers  164
446
382
inliers  204
447


309
inliers  129
799
300
inliers  103
800
296
inliers  85
801
294
inliers  90
802
295
inliers  94
803
287
inliers  77
804
297
inliers  75
805
295
inliers  106
806
284
inliers  105
807
293
inliers  108
808
294
inliers  87
809
284
inliers  107
810
311
inliers  149
811
331
inliers  160
812
331
inliers  175
813
324
inliers  185
814
345
inliers  198
815
334
inliers  194
816
325
inliers  170
817
322
inliers  186
818
337
inliers  191
819
340
inliers  213
820
341
inliers  170
821
345
inliers  176
822
336
inliers  194
823
338
inliers  173
824
346
inliers  159
825
334
inliers  168
826
315
inliers  133
827
321
inliers  148
828
312
inliers  129
829
316
inliers  98
830
315
inliers  116
831
294
inliers  100
832
293
inliers  97
833
301
inliers  117
834
296
inliers  125
835
305
inliers  124
836
314
inliers  117
837
328
inliers  128
838
297
inliers  75
839
310
inliers  89
840
295
inliers  66
841
295
inliers  61
842
288
inliers  73
843
298
inliers  120
844
317
inliers  122
845
308
inliers  109
846
306
i

Description:

First, we found correspondances between two images, estimated the $F$ matrix, and then estimated $E$ using $F$ by,

$E=K^{T}FK$

Then we decomposed $E$ into $R$ and $t$.

For running the code here, we have used sequence 06 given in 'more-sequences' for the output (which has also been attached in the folder).RANSAC was tuned to match the groundtruth 




